# Python Design Patterns

## General Knowledge
### types of design patterns
* Creational Patterns
  + used to create objects systematically
  + benefits: provides flexibility when creating objects, for example, different subtypes of objects from the same class can be created at runtime
  + polymorphism is often used
* Structural Patterns
  + establish relationships between software components to assemble classes and their instances
  + accomplish both functioal and nonfunctional goals
  + different goals will use different structures
  + take advantage of inheritance 
* Behavioral Patterns 
  + define object interactions to accomplish both functional and nonfunctional goals
  + the focus is to define the protocols between these objects, and make them work together to accomplish a common goal 
  + use methods and their signatures a lot
* interfaces are used in all the three types of design patterns
### Pattern context
* participants
  + classes involved to form a design pattern
  + different classes play different roles in a pattern
* quality attributes
  + nonfunctional requirements including usability, modifiability, reliability, performance, and more
  + impact the entire software (addressed by architecture solutions)
* forces
  + various factors or trade-offs to consider
  + manifested in quality attributes
  + unintended consequences
* consequences
  + worse performance
  + decision makers should consider consequences when considering to use a pattern
### Design patterns and Architecture patterns
* design patterns
  + at the class level
  + address how classes are instantiated, structured and orchestrated
  + have local impact, not affect the entire software as a whole, e.g. singleton which only restrict the number of objects instantiated from the same class
* architectural patterns
  + high level and abstract
  + at the module level
  + address global concerns about quality attributes, including security, performance, maintainability, ect.
  + have global impact
    + The quality attribute that are impacted most is the maintainability
    + e.g. MVC is an architecture pattern which has direct impact on the overall quality of software
    + by separating the responsibility into three distinct modules, and limiting the changes to unaffected modules, MVC improves software modifiability
      + if a user interface needs to be changed, controller and model modules don't need to be change
* framework
  + collection of reusable software elements
  + implement design patterns
  + allow developers to reuse code through application programming interfaces (APIs)
  + many domain-specific software components such as security have commercial off-the-shelf (COTS) and open-source frameworks
    + serveral frameworks are available for Authentication and suport various programming languages
### Design best practice
* Quality attributes and design patterns
  + quality attributes include non-functional requirements
    + performance
    + security
    + maintainability
  + design patterns are more directly related to functional  buand non-functional requirements
    + adapter pattern used to connect your app to different databases, such as Oracle and MySQL fufills functional requirements
    + if consistently used, the apdaptor pattern promotes maintainability of software by containing the change in a single, isolated location instead of widespread code modifications 

## Behavioral Design Patterns

### Strategy Pattern
#### Pattern Summary
* general description
  * different strategies need to be applied to a class method/function
* use case:
  + Item class needs to apply a variety of different discount strategy
  + discount strategies are implemented as different functions
  + a strategy function is passed to the class constructor to instantiate a class object applying that strategy
* advantages:
  + open/close principle: easy to introduce new strategies without changing client code
  + isolation: isolate the specific implementation details of the algorithms from the client's code
  + encapsulation: data structure used for implementing the algorithm are completely encapsulated in Strategy class. Therefore, the implementation of an algorithm can be changed without affecting the Context class 
    + here Context class is the interface/abstract class of the strategy function
  + Run-time switch: application can switch the strategies at the run-time
* disadvantages:
  + creating extra objects: needs to create and maintain both strategy class instance and strategy function object
  + awareness among clients: need to understand the difference between strategies to apply them
  + increases the complexity: no need to implement if there are only a few algorithms to implement

#### Implement Strategy Pattern
* in this implementation, the self object is passed to strategy functions

In [14]:
# code from https://www.geeksforgeeks.org/strategy-method-python-design-patterns/
"""Strategy class"""
class Strategy:

    """Constructor function with price and discount"""

    def __init__(self, price, discount_strategy = None):
        
        """take price and discount strategy"""
        
        self.price = price
        self.discount_strategy = discount_strategy
        
    
    """A separate function for price after discount"""

    def price_after_discount(self):
        
        if self.discount_strategy:
            discount = self.discount_strategy(self)
        else:
            discount = 0
            
        return self.price - discount

    def __repr__(self):

        statement = "Price: {}, price after discount: {}"
        return statement.format(self.price, self.price_after_discount())

""" Strategy functions dedicated to On Sale Discount"""
def on_sale_discount(order):
    return order.price * 0.25 + 20

"""function dedicated to 20 % discount"""
def twenty_percent_discount(order):
    return order.price * 0.20

In [15]:
"""Cleint Code"""

"""using default strategy"""
print(Item(20000))

"""with discount strategy as 20 % discount"""
print(Item(20000, discount_strategy = twenty_percent_discount))

"""with discount strategy as On Sale Discount"""
print(Item(20000, discount_strategy = on_sale_discount))

Price: 20000, price after discount: 20000
Price: 20000, price after discount: 16000.0
Price: 20000, price after discount: 14980.0


* Implementation using dynamic type creation
  * this implementation uses the dynamic method bound to Strategy class
  * the self object is directly defined in strategy functions
  * types.MethodType() takes two arguments, the function object, and the object the function is bound to

In [16]:
import types #Import the types module

class Strategy:
    """The Strategy Pattern class"""
    
    def __init__(self, price, function=None):
        self.price = price
        
        #If a reference to a function is provided, replace the execute() method with the given function
        if function:
            self.execute = types.MethodType(function, self)
            
    def execute(self): #This gets replaced by another version if another strategy is provided.
        """The defaut method that prints the name of the strategy being used"""        
        return self.price
    
    def __repr__(self):
        statement = "Price: {}, price after discount: {}"
        return statement.format(self.price, self.execute())

#Replacement method 1
def on_sale_discount(self):
    return self.price * 0.25 + 20    

#Replacement method 2    
def twenty_percent_discount(self):
    return self.price * 0.20

In [17]:
"""Cleint Code"""

"""using default strategy"""
print(Item(20000))

"""with discount strategy as 20 % discount"""
print(Item(20000, discount_strategy = twenty_percent_discount))

"""with discount strategy as On Sale Discount"""
print(Item(20000, discount_strategy = on_sale_discount))

Price: 20000, price after discount: 20000
Price: 20000, price after discount: 16000.0
Price: 20000, price after discount: 14980.0


* Template for Strategic Pattern

In [18]:
# from https://github.com/LinkedInLearning/python-design-patterns-2422610/blob/04_08e/strategy_final.py
import types #Import the types module

class Strategy:
    """The Strategy Pattern class"""
    
    def __init__(self, function=None):
        self.name = "Default Strategy"
        
        #If a reference to a function is provided, replace the execute() method with the given function
        if function:
            self.execute = types.MethodType(function, self)
            
    def execute(self): #This gets replaced by another version if another strategy is provided.
        """The defaut method that prints the name of the strategy being used"""
        print("{} is used!".format(self.name))

#Replacement method 1
def strategy_one(self):
    print("{} is used to execute method 1".format(self.name))

#Replacement method 2    
def strategy_two(self):
    print("{} is used to execute method 2".format(self.name))
    
#Let's create our default strategy
s0 = Strategy()
#Let's execute our default strategy
s0.execute()

#Let's create the first varition of our default strategy by providing a new behavior
s1 = Strategy(strategy_one)
#Let's set its name
s1.name = "Strategy One"
#Let's execute the strategy
s1.execute()

s2 = Strategy(strategy_two)
s2.name = "Strategy Two"
s2.execute()


Default Strategy is used!
Strategy One is used to execute method 1
Strategy Two is used to execute method 2


### Observer Pattern
#### Pattern summary
* general description
  * allows you to define or create a subscription mechanisms to send notifications to multiple objects about any new event happens to the object they are observing.
  * the subject maintains the sinble copy of the state information observed by multiple observers, and notify them once the state information is changed or updated.
  * this pattern defines one to many dependencies between subject and observer objects
  * this is also called sub/pub because subject publish updateds of state information to observers
  + singleton is relaated to this pattern
* use case
  + there is a core temperature of reactors at a powerplant
  + there are many registered observers that will be notified whenever temperature changes
* advantages:
  + open/closed principle: introducing new observers is easy
  + establishes relationship: easy to establish the relationships at the runtime between objects
  + loose coupling between subjects and observers. There is no need to modify subject to add or remove observers

#### Implement Observer Pattern
* define an abstact class of Subject with the following methods:
  + attach
  + detach
  + notify
* concrete subject classes inherit Subject class (Core class)
  + maintain temp property
 * Observer concrete classes implementing update() method that can be called by notify() method from concrete Subject classes 

In [21]:
class Subject(object): #Represents what is being 'observed'

    def __init__(self):
        self._observers = [] # This where references to all the observers are being kept
        # Note that this is a one-to-many relationship: there will be one subject to be observed by multiple _observers

    def attach(self, observer):
        if observer not in self._observers: #If the observer is not already in the observers list
            self._observers.append(observer) # append the observer to the list

    def detach(self, observer): #Simply remove the observer
        try:
            self._observers.remove(observer)
        except ValueError:
            pass

    def notify(self, modifier=None):
        for observer in self._observers: # For all the observers in the list
            if modifier != observer: # Don't notify the observer who is actually updating the temperature 
                observer.update(self) # Alert the observers!

class Core(Subject): #Inherits from the Subject class

    def __init__(self, name=""):
        Subject.__init__(self)
        self._name = name #Set the name of the core
        self._temp = 0 #Initialize the temperature of the core

    @property #Getter that gets the core temperature
    def temp(self):
        return self._temp

    @temp.setter #Setter that sets the core temperature
    def temp(self, temp):
        self._temp = temp
        self.notify() #Notify the observers whenever somebody changes the core temperature

class TempViewer:
    def update(self, subject): #Alert method that is invoked when the notify() method in a concrete subject is invoked
        print("Temperature Viewer: {} has Temperature {}".format(subject._name, subject._temp))

#Let's create our subjects
c1 = Core("Core 1")
c2 = Core("Core 2")

#Let's create our observers
v1 = TempViewer()
v2 = TempViewer()

#Let's attach our observers to the first core
c1.attach(v1)
c1.attach(v2)

#Let's change the temperature of our first core
c1.temp = 80
c1.temp = 90


Temperature Viewer: Core 1 has Temperature 80
Temperature Viewer: Core 1 has Temperature 80
Temperature Viewer: Core 1 has Temperature 90
Temperature Viewer: Core 1 has Temperature 90


### Iterator Pattern
#### Pattern Summary
* general description
  + Iterator Pattern is used to access elements of an aggregate object sequentially without exposing its underlying implementation
* use case
  + create a custom iterator based on built-in Python iterator: zip()
  + the iterator is used to traverse German counting words only upto a certain point
* advantages:
  + single responsibility: extract the huge algorithms in to separate classes in the iterator method so that aggregate classes don't have to also manage the iteration functions
  + open/closed principal:passing the new iterators and collections into the client code will not break the code
  + easy to use interface: it makes the interface simple to use and also supports the variations in the traversal of the collections
* disadvantages:
  + increases complexity: may introduce unnecessary complexity for simple applications
  + decreases efficiency: accessing elements directly is a much better option as compared to accessing elements using the iterator in terms of efficiency

#### Implement Iterator Pattern
* we define a generator function that will generate the German counting words, and use the for loop to take advantage of the iterator automatically provided by the for loop to traverse the generated numbers

In [25]:
def count_to(count):
    """Our iterator implementation"""
    
    #Our list
    numbers_in_german = ["eins", "zwei", "drei", "vier", "funf"]

    #Our built-in iterator
    #Creates a tuple such as (1, "eins")
    iterator = zip(range(count), numbers_in_german)
    
    #Iterate through our iterable list
    #Extract the German numbers
    #Put them in a generator called number
    for position, number in iterator:
        
        #Returns a 'generator' containing numbers in German
        yield number 

#Let's test the generator returned by our iterator
for num in count_to(3):
    print("{}".format(num))

for num in count_to(4):
    print("{}".format(num))
    

eins
zwei
drei
eins
zwei
drei
vier


### Command Pattern

#### Patern Summary
* general description
  + wraps a requests in an object for the parameterization of clients with different requests and the queuing or logging of  requests
  + it is always preferred to use command pattern when we have to parameterize the objects with the operations
  + it is used to implement reversible operations such as UNDO/REDO operations
* use case
  + different commands may work on differen receivers for different tasks
  + clients only need to construct the Command, Reciever and Invoker objects and call the Invoker's methods to process the tasks
* advantages:
  + open/closed principle: we can introduce new commands without breaking the existing client's code
  + single responsibility principle: easy to decouple the classes invoking the operations from other classes
  + implementable UNDO/REDO
  + encapsulation: it encapsulates all the information needed to perform an action or an event
* disadvantages:
  + complexity and quantity of classes increased by introducing extra layers between sender and receiver of requests
  + many concrete Command subclasses for increased code implementation and maintenance

#### Implement Command Pattern
  + Rather than directly sending the requests for different tasks, we introduce several layers
  + we first define a Command abstract class with an abstract method:process()
    + this method will be implemented by concrete Command subclasses to process their receivers
  + we define the concrete Command subclass by implementing its process() method on its receiver
  + we implement a Receiver class that can excute a specific task
  + we implement a Invoker class with two methods: 
    + command(cmd) method to set up a concrete Command instance object
    + execute() method to invoke the command object's process() method
  + in client code, we do the following:
    + construct the appropriate Reciever object
    + construct the appropriate Command object using the Receiver object
    + construct the Invoker object, and set up its Command instance using its command() function 
    + call the invoker's execute() method to process the task 

In [3]:
"""Use built-in abc to implement Abstract classes and methods"""
from abc import ABC, abstractmethod

"""Class Dedicated to Command"""
class Command(ABC):
    
    """constructor method"""
    def __init__(self, receiver):
        self.receiver = receiver
    
    """process method"""
    def process(self):
        pass

"""Class dedicated to Command Implementation"""
class CommandImplementation(Command):
    
    """constructor method"""
    def __init__(self, receiver):
        self.receiver = receiver

    """process method"""
    def process(self):
        self.receiver.perform_action()

"""Class dedicated to Receiver"""
class Receiver:
    
    """perform-action method"""
    def perform_action(self):
        print('Action performed in receiver.')

"""Class dedicated to Invoker"""
class Invoker:
    
    """command method"""
    def command(self, cmd):
        self.cmd = cmd

    """execute method"""
    def execute(self):
        self.cmd.process()

"""main method"""
if __name__ == "__main__":
    
    """create Receiver object"""
    receiver = Receiver()
    cmd = CommandImplementation(receiver)
    invoker = Invoker()
    invoker.command(cmd)
    invoker.execute()


Action performed in receiver.


### Memento Pattern

#### Pattern Summary
* general description
  + provides the ability to restore an object to its previous state.
  + allows you to save and restore the previous version of the object without revealing the details of concrete implementations
  + it tries not to disturb the encapsulation of the code and allows you to capture and externalize an object's internal state
  + according to the pattern, we should store the copy of the object's state in a special object called Memento and the content of the memento objects are not accessible to any other object except the one that produced it
  + mostly applicable to UNDO and REDO
  + another usage is to encapsulate the object's internal implementation from client's code by Memento objects rather than allowing client's code directly accessing the object
* use case
  + we used a pickle object to save the attributes of the originator object. We named the pickle object as memento
  + we then changed the self\_state attribute value of originator object, and confirmed the change
  + we then reset the attributes of originator object back to what were stored in memento object, and confirmed it  
* advantage
  + encourages encapsulation: helps producing state of object without breaking the encapsulation of the client's code
  + simplifies code by using momento object to maintain history of originator's code
  + generic Memento's implementation: better to use Serialization to achieve memento pattern implementation that is more generic rather than Memento pattern where every object needs to have it's own class implementation
* disadvatages
  + huge memory consumption: if the originator's object is very huge, then Memento object size be huge in memory
  + using generic Serialization such as Pickle can prevent the memento object being touched and difficult deletion of Memeto objects

#### Implement Memento Pattern

In [4]:
import pickle

class Originator:

    def __init__(self):
        self._state = None

    def create_memento(self):
        return pickle.dumps(vars(self))

    def set_memento(self, memento):
        previous_state = pickle.loads(memento)
        vars(self).clear
        vars(self).update(previous_state)

In [5]:
    # initialize the Originator object
    originator = Originator()

    # confirm its attribute value (instance attr)
    print(vars(originator))

    # store the attr value in Memento object
    memento = originator.create_memento()
    
    # change the originator object state value and confirm the change
    originator._state = True
    print(vars(originator))

    # restore the state value using memento object, and confirm
    originator.set_memento(memento)
    print(vars(originator))

{'_state': None}
{'_state': True}
{'_state': None}


### Interpreter Pattern

#### Pattern Summary
* general description
  + used to define a grammatical representation for a language and provides an interpreter to deal with this grammar
  + This pattern involves implementing an expression interface which tells to interpret a particular context
  + thie pattern performs upon a hierarchy of expressions. Each expression is either terminal or non-terminal
  + the tree structure of interpreter design pattern is somewhat similar to that defined by the composite design pattern with terminal expressions being leaf objects and non-terminal expressions being composite
  * the tree contains the expressions to be evaluated and is usually generated by a parser. The parser itself is not a part of the interpreter pattern  
  + This pattern is used in SQL parsing, searching patterns in text using regex, and symbol processing engine etc.
* use case
* advantage
* disadvatage

#### Implement Interpreter Pattern
* This pattern contains the following design components:
  + AbstractExpression (Expression): Declares an interpret() operation that all nodes (terminal and nonterminal) in the AST overrides
  + TerminalExpression (NumberExpression): Implements the interpret() operation for terminal expressions. It is an end of the expression and is limited to a single type
  + NonterminalExpression (AdditionExpression, SubtractExpression, and MultiplicationExpression): Implements the interpret() operation for all nonterminal expressions. It has many variations corresponding to different tasks
  + Context(String): contains information that is global to the interpreter. It is this string expression with the Postfix notation that has to be interpreted and parsed
  + Client (ExpressionParser): Builds (or is provided) the AST assembled from TerminalExpression and NonTerminalExpression. The Client invokes the interpret() operation
* In the following code example, a simple Interpreter Pattern was implemented
  + It has an AbstractExpression that forces subclasses to implement interpret() function
  + the Ternimal and NonTerminal subclasses accept an expression in dunder init() method, and their interpret() function either recursively calls the expression's interpret() function, or returns the value of the expression directly (here we just print a statement and return)

In [8]:
from abc import ABC, abstractmethod

class AbstractExpression():

    @abstractmethod
    def interpret(self):
        pass

class NonterminalExpression(AbstractExpression):

    def __init__(self, expression):
        self._expression = expression

    def interpret(self):
        print("Non-terminal expression being interpreted ...")
        self._expression.interpret()

class TerminalExpression(AbstractExpression):

    def interpret(self):
        print("Terminal expression being interpreted ...")

# client code to invoke the top level expression
ast = NonterminalExpression(NonterminalExpression(TerminalExpression()))
ast.interpret()

Non-terminal expression being interpreted ...
Non-terminal expression being interpreted ...
Terminal expression being interpreted ...


### Mediator Pattern

#### Pattern Summary
* general description
  + allows to reduce the unordered dependencies between the objects
  + objects take the help of mediator objects to communicate with each other to reduce the complexity of communications compared to individual communications, which have the following problems:
    + code complexity will increase dramatically and not maintainable
    + in paticular, a simple change in one object can have a huge ripple effect on the rest of the code (tight coupling among objects)
  + it reduces the coupling by reducing the dependcies between communicating objects
  + the mediator works as a router between objects and it can have it's own logic to provide a way of communication
* use case
  + in an airport, to avoid the collision of airplanes, intead of letting pilots communicate with each other, we need the air traffic contoller as the meditor to centrally and effectively manage the communication among individual pilots 
* advantages
  + loose coupling between colleague objects to communicate
  + decreased complexity of the code for better maintainability
* disadvatage
  + Centralization: mediator objects centralizes the control and may becomes the God Object

#### Implement Mediator Pattern
* Design components:
  + Mediator: defines the interface for communication between colleague objects
  + Concrete Mediator: implements the Mediator interface and coordinates communications between colleague objects
  + Colleague: defines the interface for communication with other colleagues
  + Concrete Colleague: implements the colleague interface and communicate with other colleagues through its mediator
* in the following code example, we demonstrated the implementation of all the 4 elements of the pattern 
  + Colleagure abstract class
    + implements the dunder init() method that defines the mediator it is assigned, and its id
    + implements the  getID() method to retrieve its id 
    + defines the abstract methods to receive and send messages for its concrete subclasses to implement
  + Concrete Colleague classes
    + implements send() and recieve() methods
    + in send() method, it first print out the message it going to send with its own id, then send the message by the meidator object assigned to it. Here the message is sent to all the other colleague objects by the mediator
    + in receive() method, it prints out the messages it recieved, with its own id
  + Mediator abatract class
    + defines the signature of add() and distribute methods
    + add() method accepts a colleague object as the input and provides the interface to register the colleague object
    + distribute() method accepts the sender and message for mediator to sent the message to colleague objects except for the sender
  + Concrete Mediatior class:
    + implement Mediator interface
    + create an internal list data structure to store the colleague objects the mediator object is assinged to
    + implement the add() method to add colleague objects to the internal list 
    + implement the distribute() method to send messages to colleague objects except for the sender

In [13]:
import sys

class Colleague(object):
    def __init__(self, mediator, id):
        self._mediator = mediator
        self._id = id

    def getID(self):
        return self._id

    def send(self, msg):
        pass

    def receive(self, msg):
        pass

class ConcreteColleague(Colleague):
    def __init__(self, mediator, id):
        super().__init__(mediator, id)

    def send(self, msg):
        print("Message '" + msg + "' sent by Colleague " + str(self._id))
        self._mediator.distribute(self, msg)

    def receive(self, msg):
        print("Message '" + msg + "' received by Colleague " + str(self._id))


class Mediator:
    def add(self, colleague):
        pass

    def distribute(self, sender, msg):
        pass

class ConcreteMediator(Mediator):
    def __init__(self):
        Mediator.__init__(self)
        self._colleague = []

    def add(self, colleague):
        self._colleague.append(colleague)

    def distribute(self, sender, msg):
        for colleague in self._colleague:
            if colleague.getID() != sender.getID():
                colleague.receive(msg)

# client code
                
mediator = ConcreteMediator()

c1 = ConcreteColleague(mediator, 1)
c2 = ConcreteColleague(mediator, 2)
c3 = ConcreteColleague(mediator, 3)

mediator.add(c1)
mediator.add(c2)
mediator.add(c3)

c1.send("Good Morning!")

Message 'Good Morning!' sent by Colleague 1
Message 'Good Morning!' received by Colleague 2
Message 'Good Morning!' received by Colleague 3


### State Pattern

#### Pattern Summary
* general description
  + allows an object to change its behavior when its internal state changes 
* use case
  + an ATM machine that transit between On and Off states
* advantages
  + Open/Close principle: we can add new states without changing client's code
  + singe responsibility principle and high cohesion: each state class will focus on its own state and transition actions
* disadvatage
  + making system complex if there are only a few states
  + each subclasses are coupled to each other for transitions

#### Implement State Pattern
* components of the pattern
 + State class: this class defines the general attributes and behaviors of State classes. It consists of  
   + name attribute defining the current state 
   + a list of allowed states for transition, 
   + state transition function such as goNext(). Usually uses if-else conditions to transit from the current to the next state
     + this method first check if the passed State class object is in the allowed list by checking the name of the class, and if so, set its own \_\_class\_\_ attribute to the passed state class object, otherwise, raise exception
 + State subclasses: Each state defines a separate class containing its state name and allowed states for transition 
 + contex class: the class whose objects change behaviors according to its state.
   + contains a state variable, and call that state object's transit method to change states
* in the following code example, we define the following classes
  + AtmState as the base class for On and Off subclasses corresponding to On and Off states of the ATM
  + On and Off state classes that only define the class variable name and allowed, which is a list of Strings
  + Context class, ATM that sets its current state as an Off state object
  + The Context class defines setState() method to switch from the current state to another
    + the state is switched by setting the state instance object's \_\_class\_\_ attribute to that of the State class objected pased to this method
* it is important to notice that the client code does not instantiate any state objects.  
  

In [16]:
class AtmState():

    name = "state"
    allowed = []

    def goNext(self, state):
        if state.name in self.allowed:
            print("Current State: ", self, " switched to: ", state.name)
            self.__class__ = state
#             print("after swithch", self.name)
        else:
            print("Current State: ", self, " switching to: ", state.name, " not possible!")

    def __str__(self):
        return self.name

class Off(AtmState):

    name = "off"
    allowed = ['on']

class On(AtmState):

    name = "on"
    allowed = ['off']

class ATM():
    
    def __init__(self):
        self.current = Off()

    def setState(self, state):
        self.current.goNext(state)

# client code
atm = ATM()

atm.setState(On)
atm.setState(Off)
atm.setState(Off)


Current State:  off  switched to:  on
after swithch on
Current State:  on  switched to:  off
after swithch off
Current State:  off  switching to:  off  not possible!


Another code example of State Pattern
* this example utilized State subclass instance objects in context object and switched among these instances
* to achieve this, the state subclass instances need to know the context object, which was passed to these instances when instantited them
* in addition to switching between different state classes, within each class, the position states are also changed using scan() method
* this implementation might be useful in real word applications, but introduced more coupling between context and state objects

In [17]:
# copied from https://www.geeksforgeeks.org/state-method-python-design-patterns/?ref=gcse
"""State class: Base State class"""
class State:

    """Base state. This is to share functionality"""

    def scan(self):
        
        """Scan the dial to the next station"""
        self.pos += 1

        """check for the last station"""
        if self.pos == len(self.stations):
            self.pos = 0
        print("Visiting... Station is {} {}".format(self.stations[self.pos], self.name))

"""Separate Class for AM state of the radio"""
class AmState(State):

    """constructor for AM state class"""
    def __init__(self, radio):
        
        self.radio = radio
        self.stations = ["1250", "1380", "1510"]
        self.pos = 0
        self.name = "AM"

    """method for toggling the state"""
    def toggle_amfm(self):
        print("Switching to FM")
        self.radio.state = self.radio.fmstate

"""Separate class for FM state"""
class FmState(State):

    """Constriuctor for FM state"""
    def __init__(self, radio):
        self.radio = radio
        self.stations = ["81.3", "89.1", "103.9"]
        self.pos = 0
        self.name = "FM"

    """method for toggling the state"""
    def toggle_amfm(self):
        print("Switching to AM")
        self.radio.state = self.radio.amstate

"""Dedicated class Radio"""
class Radio:

    """A radio. It has a scan button, and an AM / FM toggle switch."""

    def __init__(self):
        
        """We have an AM state and an FM state"""
        self.fmstate = FmState(self)
        self.amstate = AmState(self)
        self.state = self.fmstate

    """method to toggle the switch"""
    def toggle_amfm(self):
        self.state.toggle_amfm()

    """method to scan """
    def scan(self):
        self.state.scan()

""" main method """
if __name__ == "__main__":

    """ create radio object"""
    radio = Radio()
    actions = [radio.scan] * 3 + [radio.toggle_amfm] + [radio.scan] * 3
    actions *= 2

    for action in actions:
        action()


Visiting... Station is 89.1 FM
Visiting... Station is 103.9 FM
Visiting... Station is 81.3 FM
Switching to AM
Visiting... Station is 1380 AM
Visiting... Station is 1510 AM
Visiting... Station is 1250 AM
Visiting... Station is 1380 AM
Visiting... Station is 1510 AM
Visiting... Station is 1250 AM
Switching to FM
Visiting... Station is 89.1 FM
Visiting... Station is 103.9 FM
Visiting... Station is 81.3 FM


### Template Pattern

#### Pattern Summary
* general description
  + defines the skeleton of the operation and leaves the details to be implemented by the child class. 
  + Its subclasses can override the method implementations as per need but the invocation is to be in the same way as defined by an abstract class
  + this pattern is impose stricter polymorphism in that only some of the methods can be overriden, and there are some protected methods that can not be overriden by the subclasses
  + as a result fo the stricter polymorphism, we can avoid widespread variants among similar software components 
  + this pattern is highly used in framework development as they allow us to reuse the single piece of code at different places by making certain changes, and thus, avoids code duplication
* use case
  + most commonly used in pipelines where the entire process of organizing and invoking sub-processes are pre-defined, but each sub-process will have different implementations for specific pipelines implemented by subclasses  
* advantages
  + reduce duplicate code: we can put code common to subclasses in the base class
  + flexibility: subclasses can decide how to implement the steps of the algorithms
  + reuse code: this is done by the inheritance of the pattern 
* disadvatage
  + complex code: the code may become too complex to read and understand
  + different implementations of subclass for some of the methods may violate Liskov Substitution principle

#### Implement Template Pattern
* in the following code example, we put the step that is common to all subclasses in \_\_always\_do\_this() as a private method not supposed to be overriden
* for methods expected to be overriden, we define them as abstract methods using @abstractmethod decorator
* finally the execution of these methods must be in the same order for all subclasses, which is defined in the template\_method() of base class

In [18]:
import sys

from abc import ABC, abstractmethod

class AbstractClass(ABC):
#This class inherit from Abstract Base Class to allow the use of the @abstractmethod decorator
    
    def template_method(self):
        """Ths is the template method that contains a collection of 
        methods to stay the same, to be overriden, and to be overriden optionally.
        """

        self.__always_do_this()
        self.do_step_1()
        self.do_step_2()
        self.do_this_or()

    def __always_do_this(self):
        #This is a protected method that should not be overriden.

        name = sys._getframe().f_code.co_name
        print('{}.{}'.format(self.__class__.__name__, name))

    @abstractmethod
    def do_step_1(self):
        #This method should be overriden
        pass

    @abstractmethod
    def do_step_2(self):
        #This method should be overriden
        pass
    def do_this_or(self):
        print('You can overide me but you do not have to')

class ConcreteClassA(AbstractClass):
#This class inherits from the Abstract class featuring the template method. 

    def do_step_1(self):
        print('Doing step 1 for ConcreteClassA ...')

    def do_step_2(self):
        print('Doing step 2 for ConcreteClassA ...')

class ConcreteClassB(AbstractClass):
#This class inherits from the Abstract class featuring the template method.

    def do_step_1(self):
        print('Doing step 1 for ConcreteClassB ...')

    def do_step_2(self):
        print('Doing step 2 for ConcreteClassB ...')

    def do_this_or(self):
        print('Doing my own business ...')

# client code

print('==ConcreteClassA==')
a = ConcreteClassA()
a.template_method()

print('==ConcreteClassB==')
b = ConcreteClassB()
b.template_method()


==ConcreteClassA==
ConcreteClassA.__always_do_this
Doing step 1 for ConcreteClassA ...
Doing step 2 for ConcreteClassA ...
You can overide me but you do not have to
==ConcreteClassB==
ConcreteClassB.__always_do_this
Doing step 1 for ConcreteClassB ...
Doing step 2 for ConcreteClassB ...
Doing my own business ...


### Visitor Pattern

#### Pattern Summary
* general description
  + allows to add new features to an existing hierarchy without changing it
  + usually used to add new operations dynamically to existing classes with minimum changes
  + works well with recursive structures lie directory trees or XML structures. The visitor object can visit each node in the recursive structure
  + use this pattern if we have to perform oprations on all the elements of the complex object like Tree
* use case
  + we have House class and two visitors: HVAC specialist, and electrician to operate on house class instances
  + we can invoke the different features of House instance dynamically by passing the different visitors to it using its accept() method
* advantages
  + Open/Closed principle: introducing new behavior to existing classes is easy. We only need to modify the visit() method of the concrete Visitor class without making changes to the existing class
  + Single Responsibility principle: multiple versions of the same behavior can be operated into the same class, depending on the visitor instance
  + update logic: if the logic of operation is updated, we only need to update visitor implementation
* disadvatage
  + lots of updates: need to update each Visitor concrete class whenever a class is added or removed from the primary hierarchy  

#### Implement Visitor Pattern
The following code example implements Visitor Pattern
+ House class is the class being visited
  + method accept() triggers the visiting operation. This method accepts a visitor instance and invoke its visit() method
  + when calling visit() method, accept() method passes itself (self) to the visit() method
  + when the visitor is an HvacSpecialist instance, visit() will call the work\_on\_hvac() method, with Visitor object itself passed to the method
  + when the visitor is an Electrician instance, it will call the work\_on\_electricity() method, wth the Visitor object itself passed to the method
+ Visitor class is an abstract class that implements the dunder str method 
+ HvacSpecialist is a concrete Visitor class that implements the visit() method. This method accepts House instance as the argument, and invoke the house instance's work_on_hvac() method by passing itself to the method
+ Electrician is a concrete Visitor class that implements the visit() method. This method accepts House instance as the argument, and invoke the house instance's work_on_electricity() method by passing itself to the method
    
    

In [23]:
class House(object): #The class being visited 
    def accept(self, visitor):
        """Interface to accept a visitor"""
        visitor.visit(self) #Triggers the visiting operation!

    def work_on_hvac(self, hvac_specialist):
        print(self, "worked on by", hvac_specialist) #Note that we now have a reference to the HVAC specialist object in the house object!

    def work_on_electricity(self, electrician):
        print(self, "worked on by", electrician) #Note that we now have a reference to the electrician object in the house object!

    def __str__(self):
        """Simply return the class name when the House object is printed"""
        return self.__class__.__name__


class Visitor(object):
    """Abstract visitor"""
    def __str__(self):
        """Simply return the class name when the Visitor object is printed"""
        return self.__class__.__name__


class HvacSpecialist(Visitor): #Inherits from the parent class, Visitor
    """Concrete visitor: HVAC specialist"""
    def visit(self, house):
        house.work_on_hvac(self) #Note that the visitor now has a reference to the house object


class Electrician(Visitor): #Inherits from the parent class, Visitor
    """Concrete visitor: electrician"""
    def visit(self, house):
        house.work_on_electricity(self) #Note that the visitor now has a reference to the house object

#Create an HVAC specialist
hv = HvacSpecialist()
#Create an electrician
e = Electrician()

#Create a house
home = House()

#Let the house accept the HVAC specialist and work on the house by invoking the visit() method
home.accept(hv)

#Let the house accept the electrician and work on the house by invoking the visit() method
home.accept(e)



House worked on by HvacSpecialist
House worked on by Electrician


### Chain of Responsibility Pattern

#### Pattern Summary
* general description
  + this is the object-oriented version of if-elif-elif-else and enables us to rearrange the condition-action blocks dynamically at the run-time.
  + it allows us to pass the requests along the chain of handlers
  + whenever any handler receives the request, it has two choices: process it or pass it to the next handler in the chain
  + this pattern aims to decouple the senders of a request from its receiver by allowing the requet to move through chained receivers until it is handled
* use case
  + we have an integer value and a set of handlers that can handle it depending which range the value is in
* advantages
* disadvatage

#### Implement Chain of Responsibility Pattern
* the following code example implements Chain of Responsibility Pattern, with the following elements:
  + Abstract handler class that defines
    + dunder init() method that accepts its successor handler and stores it in self.\_successor attribute. This is important since this allows us to build the handler chain by chaining handlers in client code (see line 34 in code example) 
    + handle() method to handle the request using its \_handler() method, if it returns True, that means the current handler can handle the request, otherwise, call its successor's handle() method to handle the request
      + this method is inherited by concrete subclasses to trigger handler chain
    + \_handle() method is an abstract method that needs to be implemented by its concrete subclasses    
  + Concrete handlers: 
    + implements the \_handler() method of base class. returns True if it can handle the request, otherwise returns None
    + if \_handler() returns None, the handle() method in the base class will call its successor's handle() method    
  + Client class
    + dunder init() method establish the concrete Handler chain, with the out most handler the first one and the inner most one as the last one, which is None
      + the DefaultHandler has no successor!(None is passed to its constructor)
    + the handler chain is store in self.handler, which is actually a Concretehandler1 instance with a DefaultHander instance as its successor
    + delegate() method accepts a list of requests, and calls the handle() method of the handler chain to process each request in a for loop
    + the handler chain will automatically forward the requests to its successor if it can't process the request, otherwise, it will process it and return True
  + client code
    + instantiate a Client instance and a request list of integers
    + call the Client instance's delegate() method to process each request by the handler chain that the Client object established

In [24]:
class Handler: #Abstract handler
    """Abstract Handler"""
    def __init__(self, successor):
        self._successor = successor # Define who is the next handler

    def handle(self, request):
        handled = self._handle(request) #If handled, stop here

        #Otherwise, keep going
        if not handled:
            self._successor.handle(request)	

    def _handle(self, request):
        raise NotImplementedError('Must provide implementation in subclass!')

class ConcreteHandler1(Handler): # Inherits from the abstract handler
    """Concrete handler 1"""
    def _handle(self, request):
        if 0 < request <= 10: # Provide a condition for handling
            print("Request {} handled in handler 1".format(request))
            return True # Indicates that the request has been handled

class DefaultHandler(Handler): # Inherits from the abstract handler
    """Default handler"""

    def _handle(self, request):
        """If there is no handler available"""
        #No condition checking since this is a default handler
        print("End of chain, no handler for {}".format(request))
        return True # Indicates that the request has been handled

class Client: # Using handlers
    def __init__(self):
        self.handler = ConcreteHandler1(DefaultHandler(None)) # Create handlers and use them in a sequence you want
                                                              # Note that the default handler has no successor

    def delegate(self, requests): # Send your requests one at a time for handlers to handle
        for request in requests:
            self.handler.handle(request)

# Create a client
c = Client()

# Create requests
requests = [2, 5, 30]

# Send the requests
c.delegate(requests)



Request 2 handled in handler 1
Request 5 handled in handler 1
End of chain, no handler for 30


## Structural Patterns

### Facade Pattern

#### Pattern Summary
* general description
  + provides a simpler unified interface to a more complex system. 
  + Facade means the face of a building or particularly an outer lying interface of a complex system, consists of several sub-systems. 
  + It provides a simplified interface to access the underlying systems with a single entry point
  + while the unified interface hides the details of the subsystems, this pattern does not prohibit the direct access the subsystems. Developers still have the flexibility to access the details of subsystems if necessary
* use case
* advantage
  + reduce complexity: reduce the complexity to access each individual subsystem
  + isolation with protections to client code: easily isolate our code from the complexity of each subsystem, and protect the client code from changing with the changing of subsystems by maintain a stable interface to access the subsystems
  + testing process: it makes the process of testing easy by providing conveninent methods to access subsystems
  + losse coupling between client and subsytems: by providing interface methods to acces subsystems, the maitainability of the code improves
* disadvatage
  + costly process: it usually is not cheap to build the pattern with reliable system

#### Implement Facade Pattern
* It consists of a Facade class and a set of subsytems it represents through its own simple and unified inteface
  + users can access the subsystems without having to know the complex interfaces of each subsystems
* The following code example implemented a simple Facade Pattern with the following components:
  + Subsystem classes: SubsystemA and SubsystemB
  + A facade class that contains
    + the instance of each subsystem
    + a unified interface method called method, that delegate to call the methods of each subsytem to coordinate the entire system  

In [11]:
class SubsystemA:

    def method1(self):
        print('SubsystemA method1 ...')
        
    def method2(self):
        print('SubsystemA method2 ...')

class SubsystemB:
    def method1(self):
        print('SubsystemB method1 ...')
        
    def method2(self):
        print('SubsystemB method2 ...')

class Facade:

    def __init__(self):
        self._subsystem_A = SubsystemA()
        self._subsystem_B = SubsystemB()

    def method(self):
        self._subsystem_A.method1()
        self._subsystem_A.method2()
        
        self._subsystem_B.method1()
        self._subsystem_B.method2()

# client code
facade = Facade()
facade.method()


SubsystemA method1 ...
SubsystemA method2 ...
SubsystemB method1 ...
SubsystemB method2 ...


### Adapter pattern
#### Pattern Summary
* general description
  + adpater an interface into another one that a client is expecting
* use case
  + we have two objects: Korean and British that implements differnt speak functions: speak_Korean() and speak_English()
  + client only want to call a uniform method: speak()
* advantages:
  + principle of single responsibility: we separate teh concrete code from the primary logic of client. Client only need to call the uniform method
  + flexibility and reusability of the code
  + less complicated class: client class is not complicated by having to use a different interface and can use ploymorphism to swap between different implementatio of adapters
  + open/close principle: we can introduce new adapter classes into the code without violating the open/closed principle
* disadvantages:
  + complexity of code: new classes, objects and interfaces are introduced
  + adaptability: most of the time, we require many adaptations with the adaptee chain to reach the compatibility we want

#### Implement Adapter Pattern
* the following cell contains the implementation using some python features
* we passed the adaptee object to the adapter class dunder init()
* we also passed the methods that need to be translated using \*\*adapted_method, where keys are the translated method names and values are the objects of the corresponding methods in the original adaptee class
* we then use \_\_dict\_\_.update(adapted_method) to update the instance object attribute dictionary
* we then define the dunder getattr() to intercept the attribute getter to retrieve the other attributes from the passed object
* notice that in line 43-44, we transfer the methods themselves to keyword argument speak without calling the methods

In [26]:
class Korean:
    """Korean speaker"""
    def __init__(self):
        self.name = "Korean"

    def speak_korean(self):
        return "An-neyong?"

class British:
    """English speaker"""
    def __init__(self):
        self.name = "British"

    #Note the different method name here!
    def speak_english(self):
        return "Hello!"	

class Adapter:
    """This changes the generic method name to individualized method names"""

    def __init__(self, object, **adapted_method):
        """Change the name of the method"""
        self._object = object

        #Add a new dictionary item that establishes the mapping between the generic method name: speak() and the concrete method
        #For example, speak() will be translated to speak_korean() if the mapping says so
        self.__dict__.update(adapted_method)

    def __getattr__(self, attr):
        """Simply return the rest of attributes!"""
        return getattr(self._object, attr)
        
#List to store speaker objects
objects = []

#Create a Korean object
korean = Korean()

#Create a British object
british =British()

#Append the objects to the objects list
objects.append(Adapter(korean, speak=korean.speak_korean))
objects.append(Adapter(british, speak=british.speak_english))


for obj in objects:
    print("{} says '{}'\n".format(obj.name, obj.speak()))


Korean says 'An-neyong?'

British says 'Hello!'



### Decorator Pattern
#### Pattern Summary
* general description
  + allows you to dynamically attach new behaviors/features to existing object without changing their structure or using subclassing
  + generall, we implement by placing these objects inside the wrapper objects that contains the behviors
  + It is much easier to implement Decorator pattern in Python because of its built-in feature
  + note that Decorator pattern is different from inheritance because the new feature is added only to that particular object, not to the entire subclass
* use case
  + we need add HTML decoration tags around the input text
  + we want to combine these decorators in all possible orders 
* advantages
  + single responsibility principle: it is easy to divide a monolithic class which implements many possible variants of behavior into several classes using Decorator Pattern
  + Runtime Responsibilities: we can easily add or remove the responsibilities from an object at runtime
  + subclassing: The decorator pattern is an alternative to subclassing. Subclassing adds behavior at compile time, and the changes affect all instances of the original class. Decorating can provide new behavior at runtime for individual objects
* disadvantages
  + complicated decorators: it can be complicated to have decorators keep track of other decorators
  + ugly configuration: large number of code of layer might make the configurations ugly


#### Implement Decorator Pattern
* the Docorator Pattern was implemented using function decorator in the following cell

In [27]:
# version 1 using function decorator without parameters
from functools import wraps

def make_blink(function):
    """Defines the decorator"""

    #This makes the decorator transparent in terms of its name and docstring
    @wraps(function)

    #Define the inner function
    def decorator():
        #Grab the return value of the function being decorated
        ret = function() 

        #Add new functionality to the function being decorated
        return "<blink>" + ret + "</blink>"

    return decorator

#Apply the decorator here!
@make_blink
def hello_world():
    """Original function! """

    return "Hello, World!"

#Check the result of decorating
print(hello_world())

#Check if the function name is still the same name of the function being decorated
print(hello_world.__name__)

#Check if the docstring is still the same as that of the function being decorated
print(hello_world.__doc__)


<blink>Hello, World!</blink>
hello_world
Original function! 


In [28]:
# version 2 using function decorator with input parameter
from functools import wraps

def make_tag(tag):    
    def make_blink(function):
        """Defines the decorator"""

        #This makes the decorator transparent in terms of its name and docstring
        @wraps(function)

        #Define the inner function
        def decorator():
            #Grab the return value of the function being decorated
            ret = function() 

            #Add new functionality to the function being decorated
            return f"<{tag}>" + ret + f"</{tag}>"

        return decorator
    return make_blink

#Apply the decorator here!
@make_tag("blink")
def hello_world():
    """Original function! """

    return "Hello, World!"

#Check the result of decorating
print(hello_world())

#Check if the function name is still the same name of the function being decorated
print(hello_world.__name__)

#Check if the docstring is still the same as that of the function being decorated
print(hello_world.__doc__)


<blink>Hello, World!</blink>
hello_world
Original function! 


### Proxy Pattern

#### Pattern Summary
* general description
  + used to postpone resource intensive objection creation unless absolutely necessary.
  + use a placeholder to create object when its creation is necessary
  + in ths pattern, clients only interact with a proxy, which is responsible for creating resource intensive objects
  + commonly used as
    + virtual proxy: in database. create several proxies
    + protective proxy: in schools to restrict website access by wifi
    + Remote proxy: when the service object is located on a remote server. Proxy passes the client request ove the network handling all the details
    + smart proxy: provide the additional security to the application by intervene specific actions whenever the object would be accessed
* use case
  + we have producer objects that only be created when absolutely necessary
  + the proxy here is an Artist object that checks if there is a producer available for the guest
* advantages
  + open/close principale: without changing the client code, we cn introduce new proxies
  + smooth service: the proxy object works even when the service object is not ready or available
  + security: proxymethod also provides he security to the system
  + performance: increases the preformance of applications by avoiding the dupplication of the objects which might be huge size and memory intensive
* disadvantage
  + slow response: service might get slow or delayed
  + layer of abstraction: introduces another layer of abstration. Might cause problems when some of the service code is accessed by client directly, some by proxy
  + increase complexity: introduces new classes and objects
  

#### Implement Proxy Pattern

In [1]:
import time

class Producer:
    """Define the 'resource-intensive' object to instantiate!"""
    def produce(self):
        print("Producer is working hard!")

    def meet(self):
        print("Producer has time to meet you now!")

class Proxy:
    """"Define the 'relatively less resource-intensive' proxy to instantiate as a middleman"""
    def __init__(self):  
        self.occupied = 'No'
        self.producer = None

    def produce(self):
        """Check if Producer is available"""
        print("Artist checking if Producer is available ...")

        if self.occupied == 'No':
            #If the producer is available, create a producer object!
            self.producer = Producer()
            time.sleep(2)

            #Make the prodcuer meet the guest!
            self.producer.meet()
            
        else:
            #Otherwise, don't instantiate a producer 
            time.sleep(2)
            print("Producer is busy!")

#Instantiate a Proxy
p = Proxy()

#Make the proxy: Artist produce until Producer is available
p.produce()

#Change the state to 'occupied'
p.occupied = 'Yes'

#Make the Producer produce
p.produce()


Artist checking if Producer is available ...
Producer has time to meet you now!
Artist checking if Producer is available ...
Producer is busy!


### Composite Pattern

#### Pattern Summary
* general summary
  + uses a tree data structure to maintain part-whole relationships. 
  + For example, using recursive tree data structure to maintain Menu>submenu>submenu elements
  + this pattern is used when you need to produce the nested structure of tree which again include the leaf objects and other object containers
* use case
  + creating Menu and Submenu items
  + Submenu can still have Submenu items
  + display Menu and Submenu items using Composite Design Pattern
* advantages:
  + open/closed principal: new composite and child elements are added without breaking existing code
  + less memory consumption and execution time: create less number of objects to save memory
  + flexibility: introduces the flexibility of the class hierarchies that can cotains bothe primitive (leaf) and complex objects such as Composite objects
* disadvantages:
  + It is hard to restrict the type of components of a composite, and rely on tree structure
  + depend on the runtime checks to apply the type constraints

#### Implement Composite Pattern  
 * the implementation consists of three major elements:
    + Component (abstract class): helps implement the default behavior for the interace common to all classes. 
    + Child (concrete class inherits Component): represents the leaf object in the composition 
    + Composite (concerte class inherits Component that manage the Child elments by adding and deleting them from a tree data structure)
 * notice that the component_function of Child classes only print its own class name
 * component_function of Composite classes iterates and recursively calls the child class's component_function
 * Each Child object is appended to a Composite object. 
 * Composite objects are appended to other Composite objects except the top level Composite object 

In [2]:
class Component(object):
    """Abstract class"""

    def __init__(self, *args, **kwargs):
        pass

    def component_function(self):
        pass

class Child(Component): #Inherits from the abstract class, Component
    """Concrete class"""

    def __init__(self, *args, **kwargs):
        Component.__init__(self, *args, **kwargs)

        #This is where we store the name of your child item!
        self.name = args[0]

    def component_function(self):
        #Print the name of your child item here!
        print("{}".format(self.name))

class Composite(Component): #Inherits from the abstract class, Component
    """Concrete class and maintains the tree recursive structure"""

    def __init__(self, *args, **kwargs):
        Component.__init__(self, *args, **kwargs)

        #This is where we store the name of the composite object
        self.name = args[0]

        #This is where we keep our child items
        self.children = []

    def append_child(self, child):
        """Method to add a new child item"""
        self.children.append(child)

    def remove_child(self, child):
        """Method to remove a child item"""
        self.children.remove(child)

    def component_function(self):

        #Print the name of the composite object
        print("{}".format(self.name))

        #Iterate through the child objects and invoke their component function printing their names
        for i in self.children:
            i.component_function()

#Build a composite submenu 1
sub1 = Composite("submenu1")

#Create a new child sub_submenu 11
sub11 = Child("sub_submenu 11")
#Create a new Child sub_submenu 12
sub12 = Child("sub_submenu 12")

#Add the sub_submenu 11 to submenu 1
sub1.append_child(sub11)
#Add the sub_submenu 12 to submenu 1
sub1.append_child(sub12)

#Build a top-level composite menu
top = Composite("top_menu")

#Build a submenu 2 that is not a composite
sub2 = Child("submenu2")

#Add the composite submenu 1 to the top-level composite menu
top.append_child(sub1)

#Add the plain submenu 2 to the top-level composite menu
top.append_child(sub2)

#Let's test if our Composite pattern works!
top.component_function()


top_menu
submenu1
sub_submenu 11
sub_submenu 12
submenu2


### Bridge Pattern

#### Pattern Summary
* general description
  + helps untangle an unnecessarily complicated class hierarchy especially when implementation-specific classes are mixed with implementation independent classes  
  + in a typical scenario, there are two parallel, unrelated or orthogoanl abstractions
    + one is implementation specific
    + the other is implementation independent
  + this pattern also applies when different hierarchies classes exist, we can implement different hierarchies of classes, and integrate them using composing
  + our solution is to avoid abstracting both implementation-specific and implementation-dependent abstraction classes in a single class hierarchy  
* use case
  + in the specific use case of a Circle object, there are two parallel abstractions:
    + implementation-independent circle abstaction
      + define the properties of a circle and scaling it
    + implementation-dependent circle abstraction
      + how to draw it
  + our solution is to separate the abstraction into two different class hierarchies, as shown in the code example  
* advantages
  + single responsibility principle: decouples the different abstractions. Each class hierarchy focus on one abstraction
  + Open/Closed principle: we can introduce new abstractions, which is implemented independent of other abstractions
  + can be easily applied to implementation of the platform-independent features
* disadvatage
  + complexity: introduces more classes since each abstraction is implemented by a separate hierarchy of classes
  + double indirection: needs to pass messages and inmplementation for the operation to get executed, which may impact performance

#### Implement Bridge Pattern
* the following code example implements Bridge Patten for Circle class
* there are two parallel abstractions: implementation-specific and implementation-independent
  + implementation-specific abstraction involves two different ways of drawing the circle and is implemented by
    + DrawingAPIOne and DrawingAPITwo classes
  + implementation-independent abstraction involves the properties of circle itself, and scaling, and is implemented by
    + Circle class itself defines the circle's x, y and raidus properties
    + Circle class implements scale() method, which is implementation-independent
    + Circle class implements draw() method that utilize the implementation-specific class instances to draw cicles
    + the way Circle class integrates implementation-specific abstraction is by composing an implementation-specific class instance from its dunder init() method

In [22]:
class DrawingAPIOne(object):
    """Implementation-specific abstraction: concrete class one"""
    def draw_circle(self, x, y, radius):
        print("API 1 drawing a circle at ({}, {} with radius {}!)".format(x, y, radius))


class DrawingAPITwo(object):
    """Implementation-specific abstraction: concrete class two"""
    def draw_circle(self, x, y, radius):
        print("API 2 drawing a circle at ({}, {} with radius {}!)".format(x, y, radius))

class Circle(object):
    """Implementation-independent abstraction: for example, there could be a rectangle class!"""

    def __init__(self, x, y, radius, drawing_api):
        """Initialize the necessary attributes"""
        self._x = x
        self._y = y
        self._radius = radius
        self._drawing_api = drawing_api

    def draw(self):
        """Implementation-specific abstraction taken care of by another class: DrawingAPI"""
        self._drawing_api.draw_circle(self._x, self._y, self._radius)

    def scale(self, percent):
        """Implementation-independent"""
        self._radius *= percent


#Build the first Circle object using API One
circle1 = Circle(1, 2, 3, DrawingAPIOne())
#Draw a circle
circle1.draw()

#Build the second Circle object using API Two
circle2 = Circle(2, 3, 4, DrawingAPITwo())
#Draw a circle
circle2.draw()

API 1 drawing a circle at (1, 2 with radius 3!)
API 2 drawing a circle at (2, 3 with radius 4!)


## Creational Pattern

### Factory Pattern

#### Pattern Summary
* general description
  + allows an interface or a class to create an object and encapsulate the creation logic 
  + objects are created without exposing the logic to client 
  + client use the same common interface to create new type of object
  + used when there are uncertainties in types of objects to use and needs to decide what classes to create at runtime
* use case
  + we have different classes of pets, including dog and cat and may add other new classes later
  + To create instances of different classes of pets, we define the factory method: get_pet() function
  + this function encapsulates the details of how the different pet instances are created, and clients only need to tell which type of pets (dog or cat) to the function, and then get a pet
* advantages
  + allows to easily add new types of products to the factory method without disturbing the client code
  + avoides tight coupling between the product classes and creator classes and objects

#### Implement Factory Pattern
* we take the advantage of Python's feature by using python dictionary to create and fetch the appropriate instances
* the factory method used the class type argument sent to the factory method to create objects of the input classes
* instead of using multiple if/else branches, we use python dictionary to store and retrieve instance objects

In [2]:
class Dog:

    """A simple dog class"""

    def __init__(self, name):
        self._name = name

    def speak(self):
        return "Woof!"

class Cat:

    """A simple cat class"""

    def __init__(self, name):
        self._name = name

    def speak(self):
        return "Meow!"

def get_pet(pet="dog"):

    """The factory method"""

    pets = dict(dog=Dog("Hope"), cat=Cat("Peace"))

    return pets[pet]

d = get_pet("dog")

print(d.speak())

c = get_pet("cat")

print(c.speak())

Woof!
Meow!


### Abstract Factory Pattern
#### Pattern Summary
* general description
  + it is built on factory pattern
  + allows you to produce a family of related objects without specifying their concrete clases, which is decided at runtime
  + provides the encapsulation of a group of individual factories
  + this pattern tryies to abstract the creation of objects depending on the logic, business, platform etc.
* use case
  + we have a pet factory, which consists of a dog factory and a cat factory
  + each of the dog and cat factory creates dog and cat objects, as well as the related products, such as dog food and cat food ect.
  + It is clear that we have a family of pet factories. We have the following components
    + abstract factory: pet factory
    + concrete factory: dog factory and cat factory
    + concrete products: dog and dog food, cat and cat food
* advantages 
  + particularly useful when client doesn't know exactly what type to create
  + easy to introduce new variants of the products without breaking the existing client code
  + products which we get from the factory are surely compatible with each other in terms of the shared code in the abstract factory class
* disadvantages:
  + complicatd code with many small files

#### Implement Abstract Factory Pattern
* different from other languages such as java where an abstract factory class is used to define all the "shared" methods, and an abstract factory method implemented in the concrete subclasses, Python doesn't need to use such an inheritance method
* in the following code example, the concrete factory instance is directly sent to the abstract factory's constructor

In [3]:
class Dog:
    """One of the objects to be returned"""

    def speak(self):
        return "Woof!"

    def __str__(self):
        return "Dog"


class DogFactory:
    """Concrete Factory"""

    def get_pet(self):
        """Returns a Dog object"""
        return Dog()

    def get_food(self):
        """Returns a Dog Food object"""
        return "Dog Food!"


class PetStore:
    """ PetStore houses our Abstract Factory """

    def __init__(self, pet_factory=None):
        """ pet_factory is our Abstract Factory """

        self._pet_factory = pet_factory


    def show_pet(self):
        """ Utility method to display the details of the objects retured by the DogFactory """

        pet = self._pet_factory.get_pet()
        pet_food = self._pet_factory.get_food()

        print("Our pet is '{}'!".format(pet))
        print("Our pet says hello by '{}'".format(pet.speak()))
        print("Its food is '{}'!".format(pet_food))


#Create a Concrete Factory
factory = DogFactory()

#Create a pet store housing our Abstract Factory
shop = PetStore(factory)

#Invoke the utility method to show the details of our pet
shop.show_pet()


Our pet is 'Dog'!
Our pet says hello by 'Woof!'
Its food is 'Dog Food!'!


### Singleton Pattern
#### Pattern Summary
* general description
  + when you allow only one object to be created from a class
  + it is like a global variable in an object-oriented way
  + Borg or monostate singleton design pattern, multiple instances are allowed, but they all share a single copy of a state  attribute
  + notice that all Python modules are singleton, and is shared by mutiple objects
* use case
  + acts as an information cached shared by multiple objects
* advantages
  + initializations: An object created by the singleton pattern is initalized only when it is requested for the first time
  + access to the object: global access to the instance of the object
  + count of instances: in singleton, method classes can't have more than one instnace
* disadvantages:
  + multithread environment: it is not easy to use in a multithread environment, because we have to take care that the multithread wouldn't create a singleton object several times
  + single responsibility principle: it is solving two problems at a single time
  + unit test process: introducing global state to the applications makes unit test difficult

#### Implement Singleton Pattern
* The first implementation is Borg pattern
* the second one uses dobule checked Locking singleton pattern
* the third one uses a static method to check if the instance has been created

In [4]:
class Borg:
    """Borg pattern making the class attributes global"""
    _shared_data = {} # Attribute dictionary

    def __init__(self):
        self.__dict__ = self._shared_data # Make it an attribute dictionary

        
class Singleton(Borg): #Inherits from the Borg class
    """This class now shares all its attributes among its various instances"""
    #This essenstially makes the singleton objects an object-oriented global variable

    def __init__(self, **kwargs):
        Borg.__init__(self)
        self._shared_data.update(kwargs) # Update the attribute dictionary by inserting a new key-value pair 

    def __str__(self):
        return str(self._shared_data) # Returns the attribute dictionary for printing

#Let's create a singleton object and add our first acronym
x = Singleton(HTTP="Hyper Text Transfer Protocol")
# Print the object
print(x) 

#Let's create another singleton object and if it refers to the same attribute dictionary by adding another acronym.
y = Singleton(SNMP="Simple Network Management Protocol")
# Print the object
print(y)


{'HTTP': 'Hyper Text Transfer Protocol'}
{'HTTP': 'Hyper Text Transfer Protocol', 'SNMP': 'Simple Network Management Protocol'}


In [5]:
# Double Checked Locking singleton pattern
import threading


class SingletonDoubleChecked(object):

    # resources shared by each and every
    # instance

    __singleton_lock = threading.Lock()
    __singleton_instance = None

    # define the classmethod
    @classmethod
    def instance(cls):

        # check for the singleton instance
        if not cls.__singleton_instance:
            with cls.__singleton_lock:
                if not cls.__singleton_instance:
                    cls.__singleton_instance = cls()

        # return the singleton instance
        return cls.__singleton_instance


# main method
if __name__ == '__main__':

    # create class X
    class X(SingletonDoubleChecked):
        pass

    # create class Y
    class Y(SingletonDoubleChecked):
        pass

    A1, A2 = X.instance(), X.instance()
    B1, B2 = Y.instance(), Y.instance()
    assert A1 is not B1
    assert A1 is A2
    assert B1 is B2

    print('A1 : ', A1)
    print('A2 : ', A2)
    print('B1 : ', B1)
    print('B2 : ', B2)


A1 :  <__main__.X object at 0x000001B611D8A5C0>
A2 :  <__main__.X object at 0x000001B611D8A5C0>
B1 :  <__main__.Y object at 0x000001B611D88730>
B2 :  <__main__.Y object at 0x000001B611D88730>


In [7]:
# classic implementation of Singleton Design pattern
class Singleton:

    __shared_instance = 'GeeksforGeeks'

    @staticmethod
    def getInstance():
        """Static Access Method"""
        if Singleton.__shared_instance == 'GeeksforGeeks':
        	Singleton()
        return Singleton.__shared_instance

    def __init__(self):
        """virtual private constructor"""
        if Singleton.__shared_instance != 'GeeksforGeeks':
            raise Exception("This class is a singleton class !")
        else:
            Singleton.__shared_instance = self


# main method
if __name__ == "__main__":

    # create object of Singleton Class
    obj = Singleton()
    print(obj)

    # pick the instance of the class
    obj = Singleton.getInstance()
    print(obj)


### Prototype Pattern

#### Pattern Summary
* general description
  + this pattern creates object by cloning rather than building
  + it is used to create many identical objects individually that might be expensive in terms of compuating power
  + cloning is a good option because it makes a carbon copy in the memory space instead of building each individual objects from scratch
* use case
  + if we need to use many identical Python objects, we can clone the objects by making copies of a prototype object instead of building them through constructors
  + can be used to implement new objects without depending on the concrete implementation of the class
  + may also be useful to solve recurring and complex software development problems
* advantages
  + more efficient to create many identical objects by cloning rather than building them
  + less number of subclasses to code, since we can create new objects by cloning and then modify it
* disadvatage
  + may expose the implementation details of the class to use the pattern 

#### Implement Prototype Pattern
* the general strategy is to create a prototypical instance first, and then clone it whenever you need a replica
* The following code example has two classes, Prototype and Car, as described in the following:
  + Prototype: This class is responsible for cloning and return other class objects. It has four methods:
    + dunder init() initializes a dictionary to save object names and objects as keys and values, respectively
    + register_object() saves the name and object in the directionary as key and value, respectively
    + unregister_object() deletes the object with the given name from the dictionary
    + clone() gets a clone of the object of the given name from dictionary, updates it with kwargs attr, and returns it
  + Car is a common class that is saved and cloned by Prototype instance objects
  + client code just instantiate Prototype and Car objects, register the car object using a name, clone the object by its registered name, and print it   

In [19]:
import copy

class Prototype:
    
    def __init__(self):
        self._objects = {}
        
    def register_object(self, name, obj):
        """Register an object"""
        self._objects[name] = obj
        
    def unregister_object(self, name):
        """Unregister an object"""
        del self._objects[name]
        
    def clone(self, name, **attr):
        """Clone a registered object and update its attributes"""
        obj = copy.deepcopy(self._objects.get(name))
        obj.__dict__.update(attr)
        return obj
        
class Car:
    def __init__(self):
        self.name = "Skylark"
        self.color = "Red"
        self.options = "Ex"
        
    def __str__(self):
        return '{} | {} | {}'.format(self.name, self.color, self.options)
        
c = Car()
prototype = Prototype()
prototype.register_object('skylark',c)

c1 = prototype.clone('skylark')

print(c1)

Skylark | Red | Ex


### Builder Pattern

#### Pattern Summary
* general description
  + used as a solution to an anti-pattern called telescoping constructor 
  + telescoping constructor occurs when a developer attempts to build a complex object using an excessive number of constructors. For example, to build a car, you need to build all its complex components with many constructors
  + builder pattern brings order to the chaotic process of building such complex objects by partitioning the process into four roles"
    + Director: in charge of actually building a product to assemble components built by different methods together
    + Abstract Builder: provide all necessary interfaces required for building an object
      + usually initialize a product attribute as None from which components can be added and assembled
    + Concrete Builder: implements the interfaces defined by Abstract Builder for the specific type of product
    + Product: represent the object being built
  + unlike factory and abstract factory patterns, buildr pattern does not rely on polymorphism
  + the focus of Builder pattern is to reduce the complexity of building objects using a divide-and-conquer strategy
* use case
  + suitable for building similar prodcuts that require the same steps more efficient and elegant
  + suitable to cases that require to construct products step by step, which allows us to call the steps recursively
* advantages
  + reusability: use the same construction code for all different products
  + single responsibility principle: separate assembly code from the complex construction code for each product
* disadvatage
  + increases the code complexity by adding extra layers

#### Implement Builder Pattern
* The following code example implements Builder Pattern to build a car
  + class Builder is the Abstract Builder class. 
    + it creates a car attrbiute and assign None to it
    + it implements create\_new\_car() method, which creates a default car object with all its attributes set as None and stores it as self.car attribute  
  + class SKyLarkBuilder
    + inherits Builder class
    + provides implementations of methods used by Director class to build cars
    + these methods builds the different car components, and assigns them to self.car
      + note that self.car is initiated as None by the dunder init() of Builder base class
  + Director class 
    + accepts a conceret Builder instance in its dunder init() method
    + implements construct_car() function to actually build the car by calling the builder instance's methods
      + these methods will modify the self.car stored in the builder object, not in Director object
    + implements get\_car() method to return the car object stored in self\_builder object
* Director object organize the building processes using the methods provided by the concrete Builder class instance
  + for different products having the same process units, and each units may have different procedures, Builder Pattern can effectively organize the building process
  + It is much better than each specific type of Car develop its own building process
    + methods that are commonly to all products can be put in the base class for code reuse
    + code for assembly the componets made by differnt building methods are organized in Director class
    + both of these design reduce the code duplication
* client code
  + instantiate a concrete Builder class object
  + instantiate a Director object using the Builder object
  + call the construct\_car() method to build the car
  + obtaine the car object by Director object's get\_car() function

In [20]:
class Director():
    """Director"""
    def __init__(self, builder):
        self._builder = builder 
        
    def construct_car(self):
        self._builder.create_new_car()
        self._builder.add_model()
        self._builder.add_tires()
        self._builder.add_engine()
        
    def get_car(self):
        return self._builder.car          
 

class Builder():
    """Abstract Builder"""
    def __init__(self):
        self.car = None 
        
    def create_new_car(self):
        self.car = Car()        


class SkyLarkBuilder(Builder):
    """Concrete Builder --> provides parts and tools to work on the parts """
    
    def add_model(self):
        self.car.model = "Skylark"

    def add_tires(self):
        self.car.tires = "Regular tires"

    def add_engine(self):    
        self.car.engine = "Turbo engine"

class Car():
    """Product"""
    def __init__(self):
        self.model = None
        self.tires = None
        self.engine = None
        
    def __str__(self):
        return '{} | {} | {}'.format(self.model, self.tires, self.engine)


# client code    
builder = SkyLarkBuilder()
director = Director(builder)
director.construct_car()
car = director.get_car()
print(car)

Skylark | Regular tires | Turbo engine
